# What happens:
Files are created, send email with excel script

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import datetime
import win32com.client as win32
import calendar
from openpyxl import Workbook, load_workbook
from IPython.lib.display import Audio
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.cell import Cell
from openpyxl.styles import Color, Font, PatternFill

### Create functions and global vars

In [2]:
rd = 0
# Insert the directory where to save the file
# The Reporting Date will be requested and saved in a global var
def get_rd(directory):
    os.chdir(directory)
    global rd
    rd = str(input("Please enter the RD (YYYYMMDD): "))
    
cno_user = ""
cno_pwd = ""
# Input the password when it's asked
# It will be used later for the connection
def get_pwd():
    # Getting user from env
    global cno_user
    cno_user = "CNO_" + getpass.getuser()

    global cno_pwd
    # Prompting for pwd
    print("Please, enter the password for your user: ", cno_user)
    cno_pwd = getpass.getpass()
    if(cno_pwd == ""):
        print("Error - password entered as blank")
    else:
        print("Password entered")

# Insert the workspace in the parameter
# The connection will be made
def connect_to_DB(ws):
    #Setting up RAY connection:
    connection = create_engine(str("oracle://:@OCRPN"))
    connection.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                     reporting_date = rd, partition = ws))
    
# Insert the parameters required
# File_name -> the name of the file you want to save
# Sheet_name -> the name of the sheets that will be created
# Ws_nr -> the number of the respective WS that the connection will be made with
# Sqls -> the sqls that will be run
def save_file(file_name, sheet_name, ws_nr, sqls):
    # Create file
    wb = Workbook()
    for index, sheet in enumerate(sheet_name):
        wb.create_sheet(sheet)
        
    std = wb.get_sheet_by_name('Sheet')
    wb.remove_sheet(std)
    wb.save(filename = file_name)
    
    dfs = []
    # Select SQLs
    for index, sheet in enumerate(sheet_name):
        try:
            connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'), {partition})".format(
                        reporting_date = rd, partition = ws_nr[index]))
            df_temp= pd.read_sql_query( sqls[index], connection_RAY)
            dfs.append(df_temp)
        except:
            dfs.append(pd.DataFrame())
            print( str(ws_nr[index]) + " is not created")
            
    writer = pd.ExcelWriter(file_name)
    for index, df in enumerate(dfs):
        try:
            df.to_excel(writer, index=False, sheet_name = sheet_name[index])
        except:
            print()
    
    writer.save()
    print('DataFrame is written to Excel File successfully.')

def send_email(to, cc, sub, body, file_name):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = to
    mail.Cc = cc
    mail.Subject = sub
    mail.Body = body

    # To attach a file to the email (optional):
    attachment  = directory + "/" + file_name
    mail.Attachments.Add(attachment)

    mail.Display()

In [3]:
file_name = 'check_rwa_uncons_rio_prep.xlsx'
rd = ['20210930', '20211031', '20211130', '20211231', '20220131', '20220228', '20220331', '20220430', '20220531', '20220630', '20220731', '20220831', '20220930']
dire = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports/Equity Report"

def check_rwa(file_name, rd, dire):
    os.chdir(dire)
    # Create file
    wb = Workbook()
    # create excel writer
    writer = pd.ExcelWriter(file_name)
    
    dfs = []
    # Select SQLs
    for rd_item in rd:
        try:
            connection_RAY = create_engine(str("oracle://:@OCRPREP"))
            connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'), {partition})".format(
                        reporting_date = rd_item, partition = 201))
            df_temp= pd.read_sql_query( "select count(contract_reference), sum(rwa) from rzb_cdr", connection_RAY)
            dfs.append(df_temp)
        except:
            dfs.append(pd.DataFrame())
            print( "WS is not created")
            
    #writer = pd.ExcelWriter(file_name)
    for index, df in enumerate(dfs):
        try:
            print("writing")
            print(index)
            df.to_excel(writer, index=False, sheet_name = str(rd[index]))
        except:
            print("wrror")
    
    # save the excel file
    writer.save()
    print('DataFrame is written to Excel File successfully.')
    
check_rwa(file_name, rd, dire)

writing
0
writing
1
writing
2
writing
3
writing
4
writing
5
writing
6
writing
7
writing
8
writing
9
writing
10
writing
11
writing
12
DataFrame is written to Excel File successfully.


In [3]:
def send_email(to, cc, sub, body):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = to
    mail.Cc = cc
    mail.Subject = sub
    mail.Body = body


    mail.Display()

### Execute the functions

In [5]:
# temp vars
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports"

#functions are called
get_rd(directory)

Please enter the RD (YYYYMMDD): 20221031


# Missing Locacl Accounting Data

In [6]:
# define vars for email
to = ''
cc = ''


In [7]:
contacts_nwu = {
    'EQCBZ': {
        'to': 'milos.chvojka@rb.cz;marie.bukova@rb.cz',
        'cc': 'markus.hochleitner@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'Kathrein': {
        'to': 'silvia.ruzicka-kirstein@kathrein.at;jana.skalickova@kathrein.at;andreas.walcher@kathrein.at;sabrina.hammerschmid@kathrein.at',
        'cc': 'martin.berger@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RBBG': {
        'to': 'mariana.vasileva@raiffeisen.bg;hristo.marlov@raiffeisen.bg;panayot.zarov@raiffeisen.bg;nikola.iliev@raiffeisen.bg;maria.georgievva@raiffeisen.bg;Lyudmila.GENOVA@raiffeisen.bg;',
        'cc': 'martin.berger@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RBCN': {
        'to': 'Yingying.guo@cn.rbinternational.com;jana.zhang@cn.rbinternational.com',
        'cc': 'markus.hochleitner@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RBIPL': {
        'to': 'Elzbieta.SZYMANSKA@raiffeisen.pl;Adam.OKONIEWSKI@raiffeisen.pl',
        'cc': 'zoltan.szebenyi@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RBSG': {
        'to': 'carol.ng@sg.rbinternational.com,christopher.lim@sg.rbinternational.com,klaus.krombass@sg.rbinternational.com',
        'cc': 'markus.hochleitner@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RBSPKHR': {
        'to': 'marko.zoric@rba.hr;ana.juric@rba.hr',
        'cc': 'markus.hochleitner@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RBSPKMS': {
        'to': 'anessa.bekto@raibau.at;mathias.kajic@raibau.at',
        'cc': 'markus.hochleitner@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RBSPKRO': {
        'to': 'cristian.ciobanu@railoc.ro',
        'cc': 'markus.hochleitner@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RFACTOR': {
        'to': 'bernd.kaiser@raiffeisen-factorbank.at;christoph.igler@raiffeisen-factorbank.at;thomas.fuehringer@raiffeisen-factorbank.at',
        'cc': 'markus.hochleitner@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RKAG': {
        'to': 'Gayane.karapetian@rbinternational.com;karin.fuchs@rbinternational.com',
        'cc': 'martin.berger@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RLBG': {
        'to': 'mariana.vasileva@raiffeisen.bg;hristo.marlov@raiffeisen.bg;panayot.zarov@raiffeisen.bg;nikola.iliev@raiffeisen.bg;maria.georgievva@raiffeisen.bg;Lyudmila.GENOVA@raiffeisen.bg;',
        'cc': 'martin.berger@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RLHR': {       
        'to': 'petra.klaric@rl-hr.hr;nikola.baksaj@rl-hr.hr',
        'cc': 'martin.berger@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RSTS': {
        'to': 'pstrempek@rsts.cz',
        'cc': 'markus.hochleitner@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RWBB': {
        'to': 'manuela.spitzer@rbinternational.com;carina.imhoff@rbinternational.com',
        'cc': 'martin.berger@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'SLBY': {
        'to': 'olga.laschewskaya@priorbank.by;pavel.yacinovich@priorbank.by;marina.sherman@priorbank.by',
        'cc': 'nertila.gjakova@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    }
}

In [12]:
sub = ' - Completeness/Results Approval Process - 30.11.2022'
body = '''Dear colleagues,

As mention on RWA & IFRS 9 Production Processing for this month as well, 
since RIC is currently not available, we ask you to insert your approval here if you are finished:
https://wiki.rbinternational.com/confluence/pages/viewpage.action?pageId=505520571

If corrections are needed, please to follow the instruction defined on the link above.

Thank you.

Kind regards,
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''
#send_email(to, cc, sub, body)
for contact in contacts_nwu:
    send_email(contacts_nwu[contact]['to'], contacts_nwu[contact]['cc'], contact + sub, body)
    # send_email(contacts_nwu[contact]['to'], contacts_nwu[contact]['cc'], contact, contacts_nwu[contact]['coll1'], contacts_nwu[contact]['coll2'])

EQCBZ - Completeness/Results Approval Process - 30.11.2022
milos.chvojka@rb.cz;marie.bukova@rb.cz
Kathrein - Completeness/Results Approval Process - 30.11.2022
silvia.ruzicka-kirstein@kathrein.at;jana.skalickova@kathrein.at;andreas.walcher@kathrein.at;sabrina.hammerschmid@kathrein.at
RBBG - Completeness/Results Approval Process - 30.11.2022
mariana.vasileva@raiffeisen.bg;hristo.marlov@raiffeisen.bg;panayot.zarov@raiffeisen.bg;nikola.iliev@raiffeisen.bg;maria.georgievva@raiffeisen.bg;Lyudmila.GENOVA@raiffeisen.bg;
RBCN - Completeness/Results Approval Process - 30.11.2022
Yingying.guo@cn.rbinternational.com;jana.zhang@cn.rbinternational.com
RBDE - Completeness/Results Approval Process - 30.11.2022
markus.hochleitner@rbinternational.com
RBIPL - Completeness/Results Approval Process - 30.11.2022
Elzbieta.SZYMANSKA@raiffeisen.pl;Adam.OKONIEWSKI@raiffeisen.pl
RBSG - Completeness/Results Approval Process - 30.11.2022
carol.ng@sg.rbinternational.com,christopher.lim@sg.rbinternational.com,klaus

# Missing Comment for Local for Data

In [28]:
# define vars for email
to = ''
cc = ''
sub = ' - Reminder for Local Accounting Data (Completeness / December Load)'
body = '''Dear colleagues,

Since the deadline for completeness approval is today, this is a kind reminder to also fill up the data for the 2% deviation.


Thank you.

Kind regards,
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''
send_email(to, cc, sub, body)

In [7]:
# define vars for email
to = ''
cc = ''
sub = ' - Missing Comment for Local Accounting Data (Completeness / August Load)'
body = '''Dear colleagues,

For the cases below the comment for the local accounting data for your Unit is missing in the Ray Web Client. Could you please insert it?



Thank you.

Kind regards,
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''
send_email(to, cc, sub, body)

In [7]:
# define vars for email
to = ''
cc = ''
sub = '''RBRS - Dashboard Points (October Load)'''
body = '''Dear colleagues,

Due to the fact that your unit approved Results on the 17.11.2022 while the deadline was on the 11.11.2022, 
you will receive 0 points in the Dashboard for the Approval Process Section (October Load).

If you have any comments or questions regarding this issues, feel free to contact us.

Kind regards,
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''
send_email(to, cc, sub, body)

In [20]:
# temp vars
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports"
ws = 2

#functions are called
get_rd(directory)
get_pwd()
connect_to_DB(ws)

# defined vars for the save_file function 
file_name = rd + '_RLCZ_RLRE.xlsx'
sheet_name = [rd + "  RLCZ_GRP_FMA_STD", rd + " ALL_IRB", rd + "UNCONS" ]
ws_nr = [171, 192, 194]
sqls = ["""select * from rzb_cdr where booking_company in ('RLRE','RLREELEN','RLREJOTA','RLRETE','RLRESR','RLCZ')	""",
       """select * from rzb_cdr where booking_company in ('RLRE','RLREELEN','RLREJOTA','RLRETE','RLRESR','RLCZ')	""",
       """select * from rzb_cdr where booking_company in ('RLRE','RLREELEN','RLREJOTA','RLRETE','RLRESR','RLCZ')	"""]

# function is called
save_file(file_name, sheet_name, ws_nr, sqls)

# define vars for email
to = 'bernadette.dettmer@uk.rbinternational.com; devon.chaplin@uk.rbinternational.com'
cc = 'flori.luci@rbinternational.com'
sub = 'RBUK Ray Data ' + rd
body = '''Dear Karolina,

Please find attached, the RBUK data for ''' + rd +'''

Kr
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''
send_email(to, cc, sub, body, file_name)

Please enter the RD (YYYYMMDD): 20220630
Please, enter the password for your user:  CNO_WZHARBC
········
Password entered
Connection successfully
171 is not created
192 is not created
194 is not created


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).


DataFrame is written to Excel File successfully.


TypeError: send_email() takes 4 positional arguments but 5 were given

In [7]:
# define vars for email
to = ''
cc = ''
sub = ' - Missing Local Accounting Data (Completeness/ October Load)'
body = '''Dear colleagues,

For the cases below, the Local Accounting Data in the Ray Web Client is missing. 
Can you insert the Data today please?


 
Thank you.
  
Mit freundlichen Grüßen / Best Regards

Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''
send_email(to, cc, sub, body)

In [ ]:
contacts_nwu = {
    'TBSK': {
        'to': 'ivana_popovicova@tatrabanka.sk;michaela_rybova@tatrabanka.sk',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'TLSK': {
        'to': 'jozef_rak@tatrabanka.sk',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RBBH': {
        'to': 'selma.ibisevic-dinar@raiffeisengroup.ba; lejla.niksic@raiffeisengroup.ba',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RLBH': {
        'to': 'edita.cagalj@raiffeisengroup.ba; amila.dacic@raiffeisengroup.ba',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RBBY': {
        'to': 'Elena.Rayman@priorbank.by;Yuliya.Bahvalova@priorbank.by;Olga.Laschevskaya@priorbank.by;natalya.sapezinskaya@priorbank.by;Dmitry.Kovalchuk@priorbank.by',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RLBY1': {
        'to': 'natalya.truhan@priorbank.by; natalya.vusheva@priorbank.by',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com; anna.baranova@priorbank.by',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'AVAL': {
        'to': 'stanislava.dzhygurda@aval.ua',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RBAL': {
        'to': 'Lira.Veshi@raiffeisen.al;jonida.shirokaj@raiffeisen.al',
        'cc': 'nertila.gjakova@rbinternational.com; christoph.gamauf@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Christoph Gamauf'
    },
    'RLAL': {
        'to': 'holta.balli@raiffeisen.al; anida.nako@raiffeisen.al',
        'cc': 'nertila.gjakova@rbinternational.com; christoph.gamauf@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Christoph Gamauf'
    },
    'RBRU': {
        'to': 'RBRU-RWA_Calculation@raiffeisen.ru;Ekaterina.PURTOVA@raiffeisen.ru;Nadezhda.MAZAEVA@raiffeisen.ru;Vasily.LUKYANOV@raiffeisen.ru',
        'cc': 'nertila.gjakova@rbinternational.com; christoph.gamauf@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Christoph Gamauf'
    },
    'RBRUFIN': {
        'to': 'Valentina.AYUPOVA@raiffeisen.ru;Maksim.SAPOZHNIKOV@raiffeisen.ru',
        'cc': 'nertila.gjakova@rbinternational.com; christoph.gamauf@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Christoph Gamauf'
    },
    'RBRS': {
        'to': 'srdjan.milosevic@raiffeisenbank.rs;anad.nikolic@raiffeisenbank.rs',
        'cc': 'nertila.gjakova@rbinternational.com; martin.berger@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Martin Berger'
    },
    'RLRS': {
        'to': 'miodrag.milovanovic@raiffeisen-rent.rs;ivana.boricic@raiffeisen-leasing.rs;maja.isakovic@raiffeisen-leasing.rs;tatjana.marjanov@raiffeisen-leasing.rs',
        'cc': 'nertila.gjakova@rbinternational.com; martin.berger@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Martin Berger'
    },
    'RBKO': {
        'to': 'muhamet.aliu@raiffeisen-kosovo.com; ilir.kallaba@raiffeisen-kosovo.com; etleve.grajcevci@raiffeisen-kosovo.com',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RLKO': {
        'to': 'fatos.karacica@raiffeisen-kosovo.com;fatmir.uka@raiffeisen-kosovo.com',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RFACTOR': {
        'to': 'bernd.kaiser@raiffeisen-factorbank.at;christoph.igler@raiffeisen-factorbank.at;thomas.fuehringer@raiffeisen-factorbank.at',
        'cc': 'markus.hochleitner@rbinternational.com; christoph.gamauf@rbinternational.com',
        'coll1' : 'Markus Hochleitner',
        'coll2' : 'Christoph Gamauf'
    }
}

In [8]:
# temp vars
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports"
ws = 46

#functions are called
get_rd(directory)
connect_to_DB(ws)

# defined vars for the save_file function 
file_name = 'REPO.xlsx'
sheet_name = ['test']
ws_nr = [194]
sqls = ["""select* from v_rzb_rd_calc_status"""]

# function is called
save_file(file_name, sheet_name, ws_nr, sqls)


Please enter the RD (YYYYMMDD): 20220731
194 is not created
DataFrame is written to Excel File successfully.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).


In [8]:
# define vars for email
to = ''
cc = ''
sub = ' - Comment on result - 17.11.2022'
body = '''Dear colleagues,

Please note that for the below row/s the comment is missing.
Kindly asking to insert it.



Thanks in advance and kind regards,
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''
send_email(to, cc, sub, body)